In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import sklearn
torch.manual_seed(1)

In [3]:
train_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w3-p2/train_data.csv")
test_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w3-p2/test_data.csv")
submit_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w3-p2/sample_submit.csv")
print(train_df.head())
print(test_df.head())
print(submit_df.head())

   ID  company    model  year  ...  tax   mpg  engineSize  price
0   0        2    Focus  2019  ...  145  34.5         2.3  26299
1   1        1      I10  2015  ...   20  60.1         1.0   5495
2   2        3       A3  2019  ...  145  34.5         2.0  32400
3   3        3       A6  2019  ...  145  47.9         2.0  29990
4   4        2   Fiesta  2015  ...    0  65.7         1.0   7250

[5 rows x 11 columns]
   ID  company      model  year  ...  fueltype  tax   mpg  engineSize
0   0        2        Ka+  2017  ...         4  145  57.7         1.2
1   1        2       Kuga  2013  ...         0  145  53.3         2.0
2   2        3         Q5  2016  ...         0  200  47.9         2.0
3   3        1        I10  2020  ...         4  145  56.5         1.0
4   4        0   3 Series  2014  ...         0   20  68.9         2.0

[5 rows x 10 columns]
   ID  price
0   0      1
1   1      1
2   2      1
3   3      1
4   4      1


In [4]:
# preprocessing

# Label Encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# label encoding시 train, test 모두 fit
le.fit(np.concatenate([train_df['model'],test_df['model']],0))
train_df['model']= le.transform(train_df['model'])
test_df['model'] = le.transform(test_df['model'])

train_x = np.array(train_df.drop(['ID','price'],axis=1))
train_y = np.array(train_df['price'])
test_x = np.array(test_df.drop(['ID'],axis=1))

# Scaler

# from sklearn.preprocessing import StandardScaler
# scaler=StandardScaler()
# scaler.fit(train_x)
# train_x = scaler.transform(train_x)
# test_x = scaler.transform(test_x)

train_x = torch.Tensor(train_x).cuda()
train_y = torch.Tensor(train_y).unsqueeze(1).cuda()
test_x = torch.Tensor(test_x).cuda()
print(train_x.shape)
print(train_x)
print(train_y.shape)
print(train_y)

torch.Size([17085, 9])
tensor([[2.0000e+00, 2.6000e+01, 2.0190e+03,  ..., 1.4500e+02, 3.4500e+01,
         2.3000e+00],
        [1.0000e+00, 3.4000e+01, 2.0150e+03,  ..., 2.0000e+01, 6.0100e+01,
         1.0000e+00],
        [3.0000e+00, 9.0000e+00, 2.0190e+03,  ..., 1.4500e+02, 3.4500e+01,
         2.0000e+00],
        ...,
        [4.0000e+00, 9.3000e+01, 2.0180e+03,  ..., 1.4500e+02, 4.7100e+01,
         1.5000e+00],
        [2.0000e+00, 2.6000e+01, 2.0180e+03,  ..., 1.4500e+02, 5.8900e+01,
         1.0000e+00],
        [2.0000e+00, 2.5000e+01, 2.0140e+03,  ..., 0.0000e+00, 7.6400e+01,
         1.5000e+00]], device='cuda:0')
torch.Size([17085, 1])
tensor([[26299.],
        [ 5495.],
        [32400.],
        ...,
        [10950.],
        [13295.],
        [ 7299.]], device='cuda:0')


In [5]:
w = torch.zeros((9,1),requires_grad=True,device="cuda")
b = torch.zeros(1,requires_grad=True,device="cuda")
optimizer = torch.optim.SGD([w,b],lr = 1e-5)
epochs = 10000

In [6]:
for epoch in range(epochs+1):
  optimizer.zero_grad()

  h = train_x.matmul(w) + b
  cost = torch.mean(torch.abs(h-train_y))
  # cost = torch.mean((h-train_y)**2)

  cost.backward()
  optimizer.step()
  if epoch % 1000 == 0:
    print(cost.item())

16819.763671875
6541.0947265625
6280.8798828125
6270.0546875
6266.51708984375
6263.32470703125
6260.18017578125
6257.0751953125
6253.9892578125
6250.92138671875
6247.86279296875


In [7]:
with torch.no_grad():
  predict = test_x.matmul(w) + b
submit_df['price'] = predict.cpu().detach().numpy()
print(submit_df)


          ID         price
0          0  17018.554688
1          1  11416.364258
2          2  13800.506836
3          3  18189.777344
4          4   7014.439941
...      ...           ...
16826  16826  14341.912109
16827  16827  18067.312500
16828  16828  11938.366211
16829  16829  16399.583984
16830  16830   7420.736816

[16831 rows x 2 columns]
